# Feature Selection

## Import Packages

In [12]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, mutual_info_regression

# From: https://towardsdatascience.com/time-based-cross-validation-d259b13d42b8
import timesplit as ts

# import our pipeline file [TO DO: NEEDS TO BE UPDATED TO USE OURS]
import pipeline as pline

## Import cleaned data on a Fips-date level

In [13]:
df = pd.read_csv("../Data/merged.csv", parse_dates=["date"], dtype={'fips': str})
df.head()

,state,fips,county,date,cumulative_cases,cumulative_deaths,new_cases,new_deaths,new_cases_7avg,new_deaths_7avg,...,mask_mandate,retail_rec,grocery_pharm,parks,transit,workplace,residential,new doses,cumulative doses,cases_next_week
0,IL,17001,Adams,2020-03-20,1.0,0.0,1.0,0.0,1.0,0.0,...,NaN,-33.0,11.0,NaN,-7.0,-22.0,13.0,0.0,0.0,0.0
1,IL,17001,Adams,2020-03-21,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,-55.0,-14.0,NaN,-30.0,-15.0,NaN,0.0,0.0,0.0
2,IL,17001,Adams,2020-03-22,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,-63.0,-42.0,NaN,NaN,-25.0,NaN,0.0,0.0,0.0
3,IL,17001,Adams,2020-03-23,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,-49.0,-20.0,NaN,-21.0,-29.0,14.0,0.0,0.0,0.0
4,IL,17001,Adams,2020-03-24,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,-46.0,-20.0,NaN,NaN,-31.0,15.0,0.0,0.0,0.0


## Some gentle data processing to get rid of any remaining NAs

In [14]:
# filter down to dates after 2021
date_mask = (df["date"]>='01-01-2021')
df = df[date_mask]

# drop grocery_pharm, parks, transit because they have too much missingness
df.drop(columns=["grocery_pharm", "parks", "transit", "residential"], inplace = True)

# fill na's with mean for that state that date for retail_rec, workplace [TO DO: IS THIS REASONABLE?]
for var in ["retail_rec", "workplace"]:
    df[var] = df[["state", "date", var]].groupby(["state", "date"]).transform(lambda x: x.fillna(x.mean()))

# fill missing masks with 0 because the missings are from MO, who doesn't have a mask mandate
df["mask_mandate"] = df["mask_mandate"].fillna(0)

# drop na's (should just be the last week for each fips)
df = df.dropna()

df.describe(datetime_is_numeric=True)

,date,cumulative_cases,cumulative_deaths,new_cases,new_deaths,new_cases_7avg,new_deaths_7avg,2weeksago_cases_7avg,2weeksago_deaths_7avg,total_pop,...,prev_day_adult_admit_60-69_7daysum,prev_day_adult_admit_70-79_7daysum,prev_day_adult_admit_80+_7daysum,prev_day_adult_admit_unknown_7daysum,mask_mandate,retail_rec,workplace,new doses,cumulative doses,cases_next_week
count,42988,42988.000000,42988.000000,42988.000000,42988.000000,42988.000000,42988.000000,42988.000000,42988.000000,4.298800e+04,...,42988.000000,42988.000000,42988.000000,42988.000000,42988.000000,42988.000000,42988.000000,42988.000000,4.298800e+04,42988.000000
mean,2021-03-11 12:42:50.615097600,9157.990567,155.679783,22.914984,0.534893,23.971457,0.492928,28.467316,0.565390,1.022049e+05,...,3.509328,3.432376,3.090397,0.160975,0.446404,-0.917308,-16.216714,485.561948,3.082490e+04,21.711222
min,2021-01-01 00:00:00,122.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.027000e+03,...,0.000000,0.000000,0.000000,0.000000,0.000000,-80.000000,-85.000000,0.000000,1.100000e+01,0.000000
25%,2021-02-05 00:00:00,1340.000000,23.000000,1.000000,0.000000,2.000000,0.000000,2.000000,0.000000,1.464000e+04,...,0.000000,0.000000,0.000000,0.000000,0.000000,-13.000000,-21.000000,18.000000,2.205000e+03,1.000000
50%,2021-03-12 00:00:00,2785.000000,49.000000,4.000000,0.000000,5.000000,0.000000,6.000000,0.000000,3.245000e+04,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-15.000000,80.000000,6.741000e+03,4.000000
75%,2021-04-16 00:00:00,6004.500000,109.000000,14.000000,0.000000,16.000000,0.000000,20.000000,0.000000,6.664300e+04,...,3.000000,3.000000,3.000000,0.000000,1.000000,11.619048,-9.000000,302.000000,1.960925e+04,14.000000
max,2021-05-24 00:00:00,549205.000000,10745.000000,5175.000000,277.000000,2836.000000,70.000000,2859.000000,70.000000,5.198275e+06,...,230.000000,244.000000,230.000000,21.000000,1.000000,168.000000,33.000000,37855.000000,2.242133e+06,2836.000000
std,NaN,31112.499257,606.015958,93.708026,3.795248,88.925948,2.392825,101.963368,2.595267,3.369719e+05,...,11.589170,11.299858,10.352177,1.048302,0.497125,17.589969,11.513196,1596.954634,1.024134e+05,82.477891


## Split Train Test

In [15]:
# split df into features and labels
X = df.drop(columns=["cases_next_week"])
y = df["cases_next_week"]

# split df into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

# df that's just training
df_train = pd.concat([X_train, y_train], axis=1)

## Pre-process data

In [16]:
# get list of numerical features for normalization
numerical_features = X_train.columns.tolist()[4:]
numerical_features.remove('mask_mandate')
numerical_features

# normalize numerical features
X_train, X_test = pline.normalize(X_train, X_test, numerical_features)

# one hot encode categorical variables
X_train = pline.one_hot_encode(X_train, ["state", "fips"])
X_test = pline.one_hot_encode(X_test, ["state", "fips"])

# drop county cause that's the same as fips
X_train = X_train.drop(columns=["county"])
X_test = X_test.drop(columns=["county"])

## Feature Selection

### Use Variance threshold to weed out features with zero variance
"This is one of the most simple approaches to feature selection. The scikit-learn library has a method called VarianceThreshold . This method takes a threshold value and when fitted to a feature set will remove any features below this threshold. The default value for the threshold is 0 and this will remove any features with zero variance, or in other words where all values are the same."


In [17]:
X = X_train
selector = VarianceThreshold()
print("Original feature shape:", X.iloc[:, 1:].shape)
new_X = selector.fit_transform(X.iloc[:, 1:])
print("Transformed feature shape:", new_X.shape)

Original feature shape: (34390, 387)
Transformed feature shape: (34390, 387)


Looks like there are no variables with zero variance, which is good!

### Narrow down to the top 50 features with SelectKBest

In [18]:
# get k highest scoring variables
fs = SelectKBest(score_func=mutual_info_regression, k="all")

# learn relationship from training data (drop non-numerical data)
fs.fit(X_train.drop(columns=["date"]), y_train.drop(columns=["date"]))

SelectKBest(k='all',
            score_func=<function mutual_info_regression at 0x7f83453575e0>)

In [19]:
feature_scores = pd.DataFrame({'variables': X_train.drop(columns=["date"]).columns.tolist(), 
                               'score': fs.scores_})


In [20]:
# top 50 variables
top50 = feature_scores.sort_values(by="score", ascending=False).head(50)["variables"].tolist()

In [21]:
X_train = X_train[["date"] + top50]
X_test = X_test[["date"] + top50]

In [22]:
X_train.columns

Index(['date', 'new_cases_7avg', '2weeksago_cases_7avg', 'total_pop', 'male',
       'female', 'age_35_44', 'white', 'age_45_54', 'age_55_59',
       'below_500_pov', 'cumulative_cases', 'below_400_pov', 'age_under14',
       'age_62over', 'age_65over', 'age_60_64', 'housing_units', 'age_20_24',
       'age_15_19', 'age_25_34', 'below_300_pov', 'below_185_pov',
       'below_200_pov', 'below_150_pov', 'below_125_pov', 'below_50_pov',
       'below_pov', 'male_below_pov', 'non_white', 'female_below_pov',
       'hispanic', 'black', 'asian', 'new_cases', 'other_race', 'native',
       'cumulative_deaths', 'total_adult_hospitalizations', 'p_non_white',
       'p_white', 'prev_day_adult_admit_7daysum', 'p_age_62over', 'p_black',
       'age_median', 'p_age_65over', 'retail_rec',
       'prev_day_adult_admit_60-69_7daysum',
       'prev_day_adult_admit_70-79_7daysum', 'hawaiian',
       'prev_day_adult_admit_80+_7daysum'],
      dtype='object')

In [23]:
X_train.shape

(34390, 51)

### Set up indices for time-based CV

In [24]:
# set up time-based CV indices
tscv = ts.TimeBasedCV(train_period=21,
                      test_period=7,
                      freq='days')

tscv_indices = []

for train_index, test_index in tscv.split(X_train, date_column='date'):
    tscv_indices.append((train_index, test_index))

### Perform some additional feature selection via Lasso regularization

In [25]:
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SelectFromModel

In [26]:
sel_ = SelectFromModel(LassoCV(alphas=[0.2], cv=tscv_indices, max_iter=10000))
sel_.fit(X_train.drop(columns=["date"]), y_train.drop(columns=["date"]))

SelectFromModel(estimator=LassoCV(alphas=[0.2],
                                  cv=[([7, 9, 40, 45, 52, 57, 61, 62, 69, 85,
                                        89, 113, 117, 121, 123, 128, 129, 145,
                                        146, 148, 168, 178, 179, 185, 187, 189,
                                        192, 193, 200, 227, ...],
                                       [30, 47, 53, 74, 131, 163, 166, 196, 215,
                                        253, 346, 369, 405, 411, 415, 437, 448,
                                        530, 565, 614, 630, 647, 664, 697, 700,
                                        703, 704, 717, 726, 750, ...]),
                                      ([7, 9, 30, 40, 45, 47, 52, 53, 62, 69,
                                        74, 85, 89, 113, 117, 121, 123, 128,
                                        129...
                                       [27, 49, 66, 109, 110, 143, 194, 195,
                                        211, 230, 231, 25

In [27]:
selected_feat = X_train.drop(columns=["date"]).columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

total features: 51
selected features: 11
features with coefficients shrank to zero: 39


In [28]:
final_features = ["date"] + selected_feat.tolist() 
X_train_final = X_train[final_features]
X_test_final = X_test[final_features]

In [29]:
final_features

['date',
 'new_cases_7avg',
 'white',
 'hispanic',
 'asian',
 'new_cases',
 'total_adult_hospitalizations',
 'prev_day_adult_admit_7daysum',
 'p_black',
 'p_age_65over',
 'prev_day_adult_admit_70-79_7daysum',
 'prev_day_adult_admit_80+_7daysum']

In [54]:
df[(df["state"]=="IL") & (df["date"]=="5-1-2021")].head()

,state,fips,county,date,cumulative_cases,cumulative_deaths,new_cases,new_deaths,new_cases_7avg,new_deaths_7avg,...,prev_day_adult_admit_60-69_7daysum,prev_day_adult_admit_70-79_7daysum,prev_day_adult_admit_80+_7daysum,prev_day_adult_admit_unknown_7daysum,mask_mandate,retail_rec,workplace,new doses,cumulative doses,cases_next_week
407,IL,17001,Adams,2021-05-01,8325.0,148.0,3.0,0.0,8.0,0.0,...,4.0,3.0,0.0,0.0,0.0,-12.000000,-6.000000,4.0,43509.0,11.0
814,IL,17003,Alexander,2021-05-01,464.0,11.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,13.301887,-7.480519,1.0,1443.0,0.0
1239,IL,17005,Bond,2021-05-01,2018.0,30.0,2.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,13.301887,-10.000000,6.0,9333.0,2.0
1657,IL,17007,Boone,2021-05-01,6582.0,83.0,14.0,0.0,15.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19.000000,-8.000000,73.0,33870.0,10.0
2049,IL,17009,Brown,2021-05-01,696.0,12.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,13.301887,-7.480519,0.0,2796.0,1.0


## Export final train and test sets

In [31]:
# X_train_final.to_csv("../Data/Train-Test Set/X_train.csv", index=False)
# y_train.to_csv("../Data/Train-Test Set/y_train.csv", index=False)

# X_test_final.to_csv("../Data/Train-Test Set/X_test.csv", index=False)
# y_test.to_csv("../Data/Train-Test Set/y_test.csv", index=False)

In [30]:
y_train.head()

116423     6.0
102619    13.0
63714      8.0
95950     10.0
31204      3.0
Name: cases_next_week, dtype: float64